In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [3]:
def get_companies_data(df , data):
    companies_list = data.find_all("div" , attrs = {"itemtype" : "http://schema.org/ListItem"} )
    for i in companies_list:
        #Name extraction
        name = str(i.find("h2" , attrs = {"class" : "companyCardWrapper__companyName"}).text).strip()
        rated_for = i.find_all("span" , attrs = {"class" : "companyCardWrapper__ratingValues"})

        #getting company type , count , age
        company_info = str(i.find("span" , attrs = {"class" : "companyCardWrapper__interLinking" } ).text).strip().split("|")
        try:
            company_type = company_info[0]
        except:
            company_type = np.nan
        try:
            employee_count = company_info[1]
        except:
            employee_count = np.nan
        
        for j in company_info:
            if "years old" in j:
                company_age = j
                break
            else:
                company_age = "Unknown"
        #Getting overall Comapny Rating
        company_rating = i.find("span" , attrs = {"class" : "companyCardWrapper__companyRatingValue"}).text
        
        #getting reasons why company is highly rated and critically rated
        high_rated_for = i.find("span" , attrs = {"class" : "companyCardWrapper__ratingHeader--high" } )
        critically_rated_for = i.find("span" , attrs = {"class" : "companyCardWrapper__ratingHeader--critical" })

        flag = 1
        if high_rated_for!=None:
           highly_rated_for_reasons = rated_for[0].text
        else:
            flag=0
            highly_rated_for_reasons = np.nan

        if critically_rated_for!=None :
            critically_rated_for_reasons = rated_for[flag].text
        else:
            critically_rated_for_reasons = np.nan

        #review count extraction
        reviews = i.find_all("span" , attrs = {"class" : "companyCardWrapper__ActionCount"})
        review_link = i.find("a" , attrs = {"class" : "companyCardWrapper__ActionWrapper"} )

        #getting reviewvs link and opening in new window
        #driver1 = webdriver.Edge()
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Run in headless mode
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--window-size=1920x1080")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        
        service = Service(r'C:\Users\naage\Downloads\chromedriver.exe') 
        # Update the path to your ChromeDriver
        driver1 = webdriver.Chrome(service=service, options=chrome_options)
        driver1.get(review_link["href"])
        time.sleep(5)
        href_data = BeautifulSoup(driver1.page_source, 'html.parser')
        review_type_name = href_data.find_all("p" ,attrs = {"class" : "body-medium rating-name clickable-rating underline"})
        review_type_rating = href_data.find_all("p" ,attrs = {"class" : "card-rating bold-list-header clickable"})
        driver1.quit()
        
        #Creating a dictionary
        data_dict = {"Name" : name ,
                     "Sector" : company_type ,
                     "Employee count" : employee_count,
                      "Company Age" : company_age ,  
                     "Reviews Count" : reviews[0].text }
        data_dict["Highly Rated for"] = highly_rated_for_reasons
        data_dict["Critically Rated for"] = critically_rated_for_reasons

        data_dict["Overall Company Rating"] = company_rating
        
        for j in range(len(review_type_rating)):
            data_dict[ str(review_type_name[j].text).strip()+" Rating" ] = str(review_type_rating[j].text).strip()
        
        each_df = pd.DataFrame(data_dict, index = [0])
        
        df = pd.concat((df, each_df), ignore_index = True)
    return df


In [4]:
df = pd.DataFrame()
# Set up Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

service = Service(r'C:\Users\naage\Downloads\chromedriver.exe')  # Update the path to your ChromeDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

#driver = webdriver.Edge()
# Loop through each page
for i in range(25):
    
    page_link = f"https://www.ambitionbox.com/list-of-companies?page={i}"
    driver.get(page_link)
        
    #Let the page load completely
    time.sleep(5)
    
    # Parse the page source with BeautifulSoup
    bs_data = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract mobile info (assuming you have a function `get_mobile_info`)
    df = get_companies_data(df, bs_data)
    
    driver.quit()

In [5]:
display(df)

""


In [90]:
df.to_csv('Ambition_box_Top_Companies_Data.csv',index=False, header=True)